1.使用Pandas对CBLUE榜单数据进行清洗和标签数字化

In [ ]:
# 1. 安装必要的库 (如果在Colab运行，去掉感叹号前的 #)
# !pip install transformers torch pandas scikit-learn

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

# --- 模拟真实数据 (真实场景是用 pd.read_json 读取文件) ---
# 假设我们有三个意图：0:病情诊断, 1:治疗方案, 2:就医建议
data = [
    {"text": "肚子一阵一阵疼是怎么回事", "label": "病情诊断"},
    {"text": "头晕恶心想吐是什么病", "label": "病情诊断"},
    {"text": "感冒了吃什么药好得快", "label": "治疗方案"},
    {"text": "脚扭伤了怎么消肿", "label": "治疗方案"},
    {"text": "牙疼应该挂什么科", "label": "就医建议"},
    {"text": "儿童发烧去哪个科室", "label": "就医建议"},
    {"text": "高血压的早期症状", "label": "病情诊断"},
    {"text": "布洛芬一次吃几粒", "label": "治疗方案"}
]

df = pd.DataFrame(data)

# 标签编码：把文字标签变成数字 (病情诊断 -> 0, 治疗方案 -> 1...)
le = LabelEncoder()
df['label_id'] = le.fit_transform(df['label'])
print(f"类别映射: {dict(zip(le.classes_, le.transform(le.classes_)))}")

2.构建数据集：自定义Dataset类，并用BERT Tokenizer进行阶段和补充，统一长度为64，解决长短不一的问题

In [ ]:
# 定义数据集类
class MedicalDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # 核心：分词 (把汉字变成 BERT 认识的 ID)
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True, # 加 [CLS] 和 [SEP]
            max_length=self.max_len,
            padding='max_length',    # 补 0
            truncation=True,         # 截断
            return_tensors='pt'      # 返回 PyTorch 张量
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 初始化分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

3.加载模型与训练：选用bert-base-chinese预训练模型，连接一个全连接层进行分类，使用AdamW优化器，学习率设为2e-5

In [ ]:
device = torch.device("cuda")

# 加载模型 (num_labels 要等于你的类别数，这里是 3)
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)
model.to(device)

# 准备数据加载器 (Batch Size = 2，一次喂2条数据)
dataset = MedicalDataset(df['text'].values, df['label_id'].values, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 优化器
optimizer = AdamW(model.parameters(), lr=2e-5)

# --- 训练循环 (Training Loop) ---
print(">>> 开始训练...")
model.train() # 开启训练模式
for epoch in range(3): # 训练 3 轮
    total_loss = 0
    for batch in dataloader:
        # 1. 把数据搬到 GPU
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 2. 清空梯度
        model.zero_grad()

        # 3. 前向传播 (Forward) -> 算出结果
        outputs = model(input_ids, attention_mask=mask, labels=labels)
        
        # 4. 计算损失 (Loss) -> 差距有多大
        loss = outputs.loss
        total_loss += loss.item()

        # 5. 反向传播 (Backward) -> 算出怎么改参数
        loss.backward()

        # 6. 更新参数 (Step) -> 修改参数
        optimizer.step()
        
    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(dataloader):.4f}")

4.模型预测：编写推理脚本，输入自然语言即可输出对应的医疗意图

In [ ]:
# --- 预测函数 ---
def predict(text):
    model.eval() # 开启评估模式 (固定参数，不训练了)
    with torch.no_grad(): # 不计算梯度，省内存
        # 处理输入文本
        encoding = tokenizer(
            text, 
            return_tensors='pt', 
            padding=True, 
            truncation=True, 
            max_length=64
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        # 模型预测
        outputs = model(input_ids, attention_mask=attention_mask)
        
        # 取概率最大的那个类别
        preds = torch.argmax(outputs.logits, dim=1)
        
        # 把数字变回文字 (0 -> 病情诊断)
        return le.inverse_transform(preds.cpu().numpy())[0]

# --- 测试 ---
print("\n>>> 最终测试:")
print("输入: '嗓子疼该吃什么药？'")
print("模型预测:", predict("嗓子疼该吃什么药？"))

print("输入: '去医院看牙挂什么号？'")
print("模型预测:", predict("去医院看牙挂什么号？"))